In [1]:
import requests
import json
import pandas as pd
import config
from datetime import datetime
import numpy as np
import config
import joblib
import aux_function
import random
#FILES-----------------------------
from artefatos import capacity_corretor, profissao_lista_representativa
from artefatos import MIDIA_VEICULO_lista_representativa, Listas, xgboost_loaded_model
from artefatos import lista_novas_origens, ddd, idhm, matriz, corretores
from artefatos import categoria_treino, base_fair_motor, enviados_count_corretor_redistribuicao
from time import gmtime, strftime



UnpicklingError: pickle exhausted before end of frame

In [4]:
today = str(datetime.today())[:10]
Body = {'DataHoraInicio':today+' 00:00:00', 'DataHoraFim':strftime("%Y-%m-%d %H:%M:%S", gmtime())}
response = requests.request('POST', config.URL_PLANTAO, headers=config.HEADERS, data=Body)
j_1 = json.loads(response.content)

In [17]:
timestamp = datetime.timestamp(datetime.now())
j_1x = [i['idPlantao'] for i in j_1 if ( i['DataHoraInicio']<=timestamp)]

In [42]:
j_1x = [x for x in j_1 if x['DataHoraFim']>=timestamp]

In [43]:
df =pd.DataFrame(j_1x)

In [44]:
df_filtrado=df
df_filtrado['DataHoraFim']=pd.to_datetime(df_filtrado['DataHoraFim'],unit='ms')
df_filtrado['DataHoraInicio']=pd.to_datetime(df_filtrado['DataHoraInicio'],unit='ms')
df_filtrado

,idPlantao,QuantidadesCorretores,befair,Corretores,status,DataHoraFim,CPFSupervisor,DataHoraInicio,NomeSupervisor,loginExecutor,login
0,909059148,5,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'PME', 'dia...",Em andamento,2022-01-03 21:00:00,982866500,2022-01-03 12:00:00,Flavia Machado de Souza,tana@qualicorp.com.br,vmtavares
1,1004411244,76,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'PME', 'dia...",Em andamento,2022-01-04 01:00:00,11735873810,2022-01-03 13:00:00,Marcelo Da Silva,tamoliveira@qualicorp.com.br,mardsilva
2,261838095,33,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'INTERNO', ...",Em andamento,2022-01-04 02:00:00,34818401870,2022-01-03 10:00:00,Tiago Gomes Borges de Melo,manjos@qualicorp.com.br,tiago.melo
3,1001503839,21,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'INTERNO', ...",Em andamento,2022-01-04 02:00:00,32105538827,2022-01-03 10:00:00,Mayra Mitsue Yabuta,ngsantos@qualicorp.com.br,tsena
4,998768745,24,0.02702702702702703,"[{'leadsDistribuidos': 20, 'canal': 'INTERNO',...",Em andamento,2022-01-04 03:00:00,32287939822,2022-01-03 10:00:00,ADRIANO TIMOTEO DE OLIVEIRA,spdsilva@qualicorp.com.br,adriano.oliveira
5,998769484,24,0.02702702702702703,"[{'leadsDistribuidos': 20, 'canal': 'INTERNO',...",Em andamento,2022-01-03 03:00:00,32287939822,2022-01-02 10:00:00,ADRIANO TIMOTEO DE OLIVEIRA,NaN,adriano.oliveira
6,1004482887,27,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'INTERNO', ...",Em andamento,2022-01-04 02:00:00,18503435861,2022-01-03 14:30:00,Ana Claudia da Silva Gouveia,bflorentino@qualicorp.com.br,ana.gouveia
7,1000425521,22,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'INTERNO', ...",Em andamento,2022-01-03 23:00:00,19245221873,2022-01-03 12:00:00,Tabata Cardoso de Oliveira,jcostabile@qualicorp.com.br,tcdoliveira
8,986427794,9,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'INTERNO', ...",Em andamento,2022-01-03 21:00:00,83417583004,2022-01-03 12:00:00,Daiane Vasconcellos da Costa,mdalmeida@qualicorp.com.br,clongaray
9,993615697,9,0.02702702702702703,"[{'leadsDistribuidos': 0, 'canal': 'INTERNO', ...",Em andamento,2022-01-03 21:00:00,40300599889,2022-01-03 12:00:00,Nuria Ceolly Muniz Daros,NaN,ndaros


In [ ]:


timestamp = datetime.timestamp(datetime.now())
j_1x = [i['idPlantao'] for i in j_1 if i['DataHoraFim']>=timestamp]
j_2 = [i['Corretores'] for i in j_1x if i['status']=='Em andamento']
corretores=[]
for i in j_2:
    for j in i:
        corretores.append(j)

return j_1x

In [12]:
import requests
import json
import pandas as pd
import numpy as np
import glob
import config 
url = 'http://localhost:6000/leads?apikey=031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406'

In [13]:
arq_list = config.main_path +'\\recebidos\\'+ "99569500_recebido_2022127"+".csv"
arq_list

'c:\\Users\\lumini04\\Desktop\\MOTOR_LEADS_NOVO\\recebidos\\99569500_recebido_2022127.csv'

In [14]:
df = pd.read_csv(arq_list)

In [15]:
df.head()


,DH_INTEGRACAO,OPERADORA,FL_65_ANOS,VALOR_PLANO_DEPENDENTES,EMAIL,ID_LEAD,NOME_EMPRESA,PRACA,NM_ORIGEM,UF,...,SCORE_PROPENSAO_LEAD,PROPENSAO_LEAD,LEAD_SOLICITACAO,LEAD_MOBILE,MIDIA_TERMO,MIDIA_CONTEUDO,MIDIA_CAMPANHA,LEAD_CLICK_TO_CALL,MIDIA_VEICULO,MIDIA_FORMATO
0,1643295028325,Intermédica,False,[],nininhahelena@terra.com.br,6113501,NaN,GSP,SOLICITAÇÃO BR,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1643295043083,Intermédica,False,[],sylvio.romanelli@gmail.com,6113516,NaN,GSP,SOLICITAÇÃO BR,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1643295083313,Intermédica,False,[],zoedutra@outlook.com,6113534,NaN,GSP,SOLICITAÇÃO BR,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1643295096753,Qualicorp,False,[],mariocandido@hotmail.com,6113548,NaN,GSP,SOLICITAÇÃO BR,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1643295109664,Intermédica,False,[],lilasilva1807@gmail.com,6113561,NaN,GSP,SOLICITAÇÃO BR,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = pd.read_csv(arq_list)
id_lead_cod_entrada = sum(df['ID_LEAD'])*len(df['ID_LEAD'])
body_json = df.to_dict()
body = {"leads":body_json, "flagDistribuicaoScore":False}


In [26]:

response = requests.request('POST', url, headers={'Content-Type':'application/json'}, data=json.dumps(body))


In [22]:
r = df.from_dict(response.json()).T

In [23]:
r

,CPF_CORRETOR,CPF_CORRETOR_2,CPF_CORRETOR_3,ID_LEAD,RATING,RATING_2,RATING_3,SCORE
0,3.378948e+10,3.076631e+10,3.015766e+10,6113501.0,2.290637,2.290637,2.290637,0.101585
1,3.378948e+10,3.076631e+10,3.015766e+10,6113516.0,2.290637,2.290637,2.290637,0.054539
2,3.015766e+10,3.076631e+10,3.015766e+10,6113534.0,2.290637,2.290637,2.290637,0.054539
3,3.378948e+10,3.076631e+10,3.015766e+10,6113548.0,2.290637,2.290637,2.290637,0.122933
4,3.378948e+10,3.076631e+10,3.015766e+10,6113561.0,2.290637,2.290637,2.290637,0.101585
5,3.015766e+10,3.076631e+10,3.015766e+10,6113578.0,2.290637,2.290637,2.290637,0.101585
6,3.015766e+10,3.076631e+10,3.015766e+10,6113591.0,2.290637,2.290637,2.290637,0.101585
7,3.015766e+10,3.076631e+10,3.015766e+10,6113610.0,2.290637,2.290637,2.290637,0.101585
8,3.076631e+10,3.076631e+10,3.015766e+10,6113624.0,2.290637,2.290637,2.290637,0.053637
9,3.015766e+10,3.076631e+10,3.015766e+10,6113641.0,2.290637,2.290637,2.290637,0.101585


In [ ]:
r = df.from_dict(response.json()).T
id_lead_cod_saida = sum(r['ID_LEAD'])*len(r['ID_LEAD'])
if id_lead_cod_entrada == id_lead_cod_saida:
    print('Retorno OK')

In [20]:
for arq in arq_list:
    try:
        df = pd.read_csv(arq_list)
    except:
        print('DataFrame Vazio')
    id_lead_cod_entrada = sum(df['ID_LEAD'])*len(df['ID_LEAD'])
    body_json = df.to_dict()
    body = {"leads":body_json, "flagDistribuicaoScore":False}
    
    response = requests.request('POST', url, headers={'Content-Type':'application/json'}, data=json.dumps(body))
    r = df.from_dict(response.json()).T
    id_lead_cod_saida = sum(r['ID_LEAD'])*len(r['ID_LEAD'])
    if id_lead_cod_entrada == id_lead_cod_saida:
        print('Retorno OK')

Retorno OK


In [62]:
#corretores plantao
today = str(datetime.today())[:10]
Body = {'DataHoraInicio':today+' 00:00:00', 'DataHoraFim':today+' 23:59:59'}
corretor_plantao = requests.request('POST', config.URL_PLANTAO, headers=config.HEADERS, data=Body)
# j_1 = json.loads(corretor_plantao.content)
# df =pd.DataFrame(j_1)
json.loads(corretor_plantao.content)

[{'idPlantao': 1041828377,
  'QuantidadesCorretores': 22,
  'befair': None,
  'Corretores': [{'leadsDistribuidos': 20,
    'uf': None,
    'diasSemana': [{'domingo': True},
     {'segunda-feira': True},
     {'terca-feira': True},
     {'quarta-feira': True},
     {'quinta-feira': True},
     {'sexta-feira': True},
     {'sabado': True}],
    'quantidadeRedistribuicao': 10,
    'quantidadeLeads': 30,
    'canal': 'PME',
    'leadsRedistribuido': 0,
    'enviadoIniciado': True,
    'cpf': 3416837525,
    'pausado': False},
   {'leadsDistribuidos': 24,
    'uf': None,
    'diasSemana': [{'domingo': True},
     {'segunda-feira': True},
     {'terca-feira': True},
     {'quarta-feira': True},
     {'quinta-feira': True},
     {'sexta-feira': True},
     {'sabado': True}],
    'quantidadeRedistribuicao': 10,
    'quantidadeLeads': 30,
    'canal': 'PME',
    'leadsRedistribuido': 0,
    'enviadoIniciado': True,
    'cpf': 92038638500,
    'pausado': False},
   {'leadsDistribuidos': 15,
    

In [58]:
j_1 = json.loads(corretor_plantao.content)
df =pd.DataFrame(j_1)

In [50]:
corretor_plantao

<Response [502]>

In [6]:
timestamp = datetime.timestamp(datetime.now())
j_1x = [x for x in j_1 if x['DataHoraFim']>=timestamp]
j_2 = [i['Corretores'] for i in j_1x if i['status']=='Em andamento']

In [23]:
corretores=[]
for i in j_2:
    for j in i:
        corretores.append(j)

In [26]:
corretores

[{'leadsDistribuidos': 17,
  'uf': None,
  'diasSemana': [{'domingo': True},
   {'segunda-feira': True},
   {'terca-feira': True},
   {'quarta-feira': True},
   {'quinta-feira': True},
   {'sexta-feira': True},
   {'sabado': True}],
  'quantidadeRedistribuicao': 10,
  'quantidadeLeads': 30,
  'canal': 'PME',
  'leadsRedistribuido': 0,
  'cpf': 3416837525,
  'pausado': True},
 {'leadsDistribuidos': 5,
  'uf': None,
  'diasSemana': [{'domingo': True},
   {'segunda-feira': True},
   {'terca-feira': True},
   {'quarta-feira': True},
   {'quinta-feira': True},
   {'sexta-feira': True},
   {'sabado': True}],
  'quantidadeRedistribuicao': 10,
  'quantidadeLeads': 30,
  'canal': 'PME',
  'leadsRedistribuido': 0,
  'cpf': 92038638500,
  'pausado': False},
 {'leadsDistribuidos': 17,
  'uf': None,
  'diasSemana': [{'domingo': True},
   {'segunda-feira': True},
   {'terca-feira': True},
   {'quarta-feira': True},
   {'quinta-feira': True},
   {'sexta-feira': True},
   {'sabado': True}],
  'quantid

In [27]:
df_teste = pd.DataFrame(list(map(lambda x: x.values(), corretores)))

In [30]:
df_teste['cp'] = df_teste[4]-df_teste[0]
df_teste['cpr'] = df_teste[3]-df_teste[4]

In [44]:
str(datetime.now())

'2022-02-07 17:10:51.573920'

In [60]:
from datetime import timedelta

def filtro_plantao(increment_time=5):
    today = str(dataetime.today())[:10]    
    body = {'DataHoraInicio':today+' 00:00:00', 'DataHoraFim':today+' 23:59:59'}
    response = requests.request('POST', config.URL_PLANTAO, headers=config.HEADERS, data=body)
    timestamp = datetime.timestamp(datetime.now() + timedelta(minutes=increment_time))

    output = json.loads(response.content)
    output = filter(lambda x: x['DataHoraFim'] > timestamp, output)
    output = filter(lambda x: x['status'] == 'Em andamento', output)
    
    corretores = [item for sublist in output for item in sublist['Corretores']]
    corretores = filter(lambda x: x['pausado']==False, corretores)
    
    corretor_plantao = filter(lambda x: x['canal']=='INTERNO', corretores)
    corretor_plantao_cpf = list(map(lambda x: float(x['cpf']), corretor_plantao))
    
    log_corretor_plantao = pd.DataFrame({'cpf': list(map(lambda x: x['cpf'], corretores)),
                                         'canal': list(map(lambda x: x['canal'], corretores))})
    
    log_corretor_plantao['Filtro'] =  log_corretor_plantao.cpf.apply(
        lambda x: x in corretor_plantao_cpf)

    get_value = lambda col: list(map(lambda x: x[col], corretor_plantao))
    corretor_plantao_df = pd.DataFrame({'corretor_plantao': corretor_plantao_cpf,
                                        'leadsDistribuidos': get_value('leadsDistribuidos'),
                                        'leadsRedistribuido': get_value('leadsRedistribuido'),
                                        'quantidadeLeads': get_value('quantidadeLeads'),
                                        'quantidadeRedistribuicao': get_value('quantidadeRedistribuicao'),
                                        'uf_corretor': get_value('uf')})
    
    corretor_plantao_df['capacity_restante'] = corretor_plantao_df['quantidadeLeads'] - \
        corretor_plantao_df['leadsDistribuidos']
    corretor_plantao_df['capacity_restante_redistribuicao'] = corretor_plantao_df['quantidadeRedistribuicao'] - \
        corretor_plantao_df['leadsRedistribuido']
    corretor_plantao_df = corretor_plantao_df[corretor_plantao_df.capacity_restante>0]
    corretor_plantao_df = corretor_plantao_df[corretor_plantao_df.capacity_restante_redistribuicao>0]
    
    log_corretor_plantao = pd.merge(log_corretor_plantao, corretor_plantao_df[['corretor_plantao', 'capacity_restante']],
                                    left_on='cpf', right_on='corretor_plantao', how='left')
    log_corretor_plantao = log_corretor_plantao.drop('corretor_plantao', axis=1)
    
    # tabele de corretores canal interno envida pela Jessica ------------------------------------
    corretore_canal_interno = pd.read_excel('historico_bi/consultores_canal_interno.xlsx')
    corretore_canal_interno=list(corretore_canal_interno.CPF.unique())
    corretor_plantao_df = corretor_plantao_df[corretor_plantao_df["corretor_plantao"].isin(corretore_canal_interno)]
    #------------------------------------------------------------------------------------------------------------
    log_corretor_plantao.to_csv(config.path_2+'/'+data_recebido+'_log_corretor_plantao.csv')
    return list(corretor_plantao_df['corretor_plantao'])

In [56]:
filtro_plantao()

NameError: name 'data_recebido' is not defined

In [ ]:
#corretores ?
response = requests.request("POST", config.URL_CORRETOR, headers=config.HEADERS, data={})

In [27]:

response = requests.request("POST", config.URL_BEFAIR, headers=config.HEADERS, data={})

In [28]:
response

<Response [200]>